### DAML WEEK 8 CHECKPOINT 4
Week of Nov 5, 2023

In [ ]:
#==========================#
# Import relevant packages #
#==========================#


import math 
import numpy as np
import matplotlib.pyplot as plt

### Problem 1

In [ ]:
#===========================================================================#
# First create the Linear class (used to model the background distribution) #
#===========================================================================#


class Linear(object):
    #————————————————————————————————————————————————————————#
    # Class constructor | initialize an attribute mass array #
    #————————————————————————————————————————————————————————#

    def __init__(self):

        # background para    
        self.xmin = 0.
        self.xmax = 20.
        self.intercept = 20.
        self.slope = -1.

        # signal para
        self.mean = 10.
        self.sigma = 0.5

        # chose number of bins that is appropriate for the size of the statistics sample
        self.nbins = 100

        #initialize an mass list used to store mass values
        self.mass = []


    #——————————————————————————————————#
    # Evaluation method (un-normalize) #
    #——————————————————————————————————#

    def evaluate (self, t):
        return self.intercept + self.slope * t
    

    
        